# **ETL AUSTRALIAN_USERS_ITEMS_ANALYSIS**

## 1. Importación de Librerías

In [3]:
import pandas as pd # Cargamos la libreria de "pandas" para la manipulación y el análisis de datos
import numpy as np # Cargamos la librería de "numpy" para realizar cálculos lógicos y matemáticos sobre cuadros y matrices en el caso que lo necesitemos
import chardet #Cargamos la librería "chardet" que nos permitirá saber el tipo de enconding que tiene nuestro archivo ha analizar
import gzip #Cargando la librería "json" nos permitira manipular archivos tipo JSON
import ast #Cargaremos ast para poder extraer datos de json
import warnings
warnings.filterwarnings("ignore")
import re

## 2. ETL (Extract - Transform - Load)

### 2.1. Carga de DataSet

In [1]:
#Colocamos la ruta del archivo JSON
user_items_ruta = r'Dataset\users_items.json.gz'

In [26]:
#Utilizamos la liberia "chardet" para saber la codificación del archivo
with open(user_items_ruta, 'rb') as archivo:
    resultado = chardet.detect(archivo.read())

# Imprimimos la codificación detectada
print(f"La codificación detectada es: {resultado['encoding']}")

La codificación detectada es: MacRoman


In [4]:
#Leemos el archivo JSON lo cargamos con el encoding detectado y lo guardamos a una lista "filas_reviews"
filas_items=[]
with gzip.open(user_items_ruta) as file:
    for line in file.readlines():
        line = line.decode('MacRoman')
        filas_items.append(ast.literal_eval(line))

# Se convierte en dataframe
df_user_items = pd.DataFrame(filas_items)

In [4]:
#Visualizamos los primeras 5 filas del archivo cargado en un DataFrame
df_user_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


### 2.2. Analisís Descriptivo del DataSet

#### 2.2.1. Definición de las Funciones Descriptivas del DataSet

In [5]:
#Definimos algunas funciones para que nos facilita la descripcion de las principales caracteristicas del DataFrame
def caracteristicas_df(df):
    """
    Describe de forma general la base de datos .

    Esta función simplemente muestra el tamaño, información general y
    la cantidad de datos nulos.

    Parametros
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        - 'df.shape': Numero de filas y columnas
        - 'df.info': Muestra información general del DataFrame

    """
    print('*'*10 + '|'*10 + 'FORMA DE BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(f'Tiene {df.shape[0]} filas y {df.shape[1]} columnas o variables')
    print(end = '\n'*2)

    print('*'*10 + '|'*10 + 'INFORMACION GENERAL DE LA BASE DE DATOS' + '|'*10 + '*'*10, end = '\n'*2)
    print(df.info(), end = '\n'*2)

def valores_nulos_df(df):
    """
    Revisa presencia de valores nulos en un DataFrame.
    Esta función toma un DataFrame como entrada y devuelve un resumen que incluye información sobre
    el porcentaje de valores no nulos y nulos, así como la ncantidad de valores nulos por columna.

    Parametros:
    ----------
    df (pandas.DataFrame): El DataFrame que se va a analizar.

    Returns:
    ----------
        pandas.DataFrame: Un DataFrame que contiene el resumen de cada columna, incluyendo:
        - 'nombre': Nombre de cada columna.
        - 'no_nulos_%': Porcentaje de valores no nulos en cada columna.
        - 'nulos_%': Porcentaje de valores nulos en cada columna.
        - 'nulos': Cantidad de valores nulos en cada columna.

    """
    mi_df = {"nombre": [], "tipo_datos": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_df["nombre"].append(columna)
        mi_df["tipo_datos"].append(df[columna].apply(type).unique())
        mi_df["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_df["nulos"].append(df[columna].isnull().sum())

    df_nulos = pd.DataFrame(mi_df)

    return df_nulos

#### 2.2.2. Descripcion del DataSet

In [6]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
caracteristicas_df(df_user_items)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 88310 filas y 5 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB
None



In [7]:
#Llamamos a la función creada para visualizar las caracteristicas generales del DataSet
valores_nulos_df(df_user_items)
#Notamos que la columna "items" tiene un tipo de dato lista

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,items_count,[<class 'int'>],0.0,0
2,steam_id,[<class 'str'>],0.0,0
3,user_url,[<class 'str'>],0.0,0
4,items,[<class 'list'>],0.0,0


#### 2.2.3. Revisamos algunos datos de columnas para conocer su estructura:

In [8]:
#Seleccionamos un datos aleatorio en la columna 'user_id' para ver el contenido
df_user_items['user_id'][0]

'76561197970982479'

In [9]:
#Seleccionamos un datos aleatorio en la columna 'items_count' para ver el contenido
df_user_items['items_count'][100]

83

In [10]:
#Seleccionamos un datos aleatorio en la columna 'steam_id' para ver el contenido
df_user_items['steam_id'][400]

'76561198080534850'

In [11]:
#Seleccionamos un datos aleatorio en la columna 'user_url' para ver el contenido
df_user_items['user_url'][500]

'http://steamcommunity.com/profiles/76561198088886304'

In [12]:
#Seleccionamos un datos aleatorio en la columna 'items' para ver el contenido
df_user_items['items'][1]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '80',
  'item_name': 'Counter-Strike: Condition Zero',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '100',
  'item_name': 'Counter-Strike: Condition Zero Deleted Scenes',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 220,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: Source',
  'playtime_forever': 62,
  'playtime_2weeks': 0},
 {'item_id': '280',
  'item_name': 'Half-Life: Source',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': 

Este conjunto contiene 5 columnas y 88309 filas, no tiene nulos. Las columnas que contiene son:

*   **user_id:** contiene un identificador único del usuario.
*   **items_count:** contiene un número entero que indica la cantidad de juegos que ha consumido el usuario.
*   **steam_id:** es un número único para la plataforma.
*   **user_url:** es la url del perfil del usuario.
*   **items:** contiene una lista de uno o mas diccionarios de los items que consume cada usuario. Cada diccionario tiene las siguientes claves:
    *   **item_id:** es el identificados del item, es decir, del juego.
    *   **item_name:** es el nombre del contenido que consume, es decir, del juego.
    *   **playtime_forever:** es el tiempo acumulado que un usuario jugó a un juego.
    *   **playtime_2weeks:** es el tiempo acumulado que un usuario jugó a un juego en las últimas dos semanas.

### 2.3. Transformación de Datos

#### 2.3.1. Funciones de Transformación

In [13]:
def verifica_duplicados(df, columna):
    '''
    Verifica y muestra filas duplicadas en un DataFrame basado en una columna específica.

    Esta función toma como entrada un DataFrame y el nombre de una columna específica.
    Luego, identifica las filas duplicadas basadas en el contenido de la columna especificada,
    las filtra y las ordena para una comparación más sencilla.

    Parameters:
    ----------
        df (pandas.DataFrame): El DataFrame en el que se buscarán filas duplicadas.
        columna (str): El nombre de la columna basada en la cual se verificarán las duplicaciones.

    Returns:
    ----------
        pandas.DataFrame or str: Un DataFrame que contiene las filas duplicadas filtradas y ordenadas,
        listas para su inspección y comparación, o el mensaje "No hay duplicados" si no se encuentran duplicados.
    '''
    # Se filtran las filas duplicadas
    duplicated_rows = df[df.duplicated(subset=columna, keep=False)]
    if duplicated_rows.empty:
        return "No hay duplicados"

    # se ordenan las filas duplicadas para comparar entre sí
    duplicated_rows_sorted = duplicated_rows.sort_values(by=columna)
    return duplicated_rows_sorted

#### 2.3.2. Transformación y eliminación de datos de TODO el DataFrame

In [14]:
#Verificamos si existen duplicados en la columna "user_id", si existen eliminar
filas_duplicadas = verifica_duplicados(df_user_items, 'user_id')

#Revisamos la fila de reviews si en verdad se repiten y SI se repiten
filas_duplicadas['items']

11000    [{'item_id': '4000', 'item_name': 'Garry's Mod...
29193    [{'item_id': '4000', 'item_name': 'Garry's Mod...
37062    [{'item_id': '220', 'item_name': 'Half-Life 2'...
37061    [{'item_id': '220', 'item_name': 'Half-Life 2'...
6167     [{'item_id': '240', 'item_name': 'Counter-Stri...
                               ...                        
4625     [{'item_id': '230410', 'item_name': 'Warframe'...
3473     [{'item_id': '20', 'item_name': 'Team Fortress...
34176    [{'item_id': '20', 'item_name': 'Team Fortress...
12417    [{'item_id': '18110', 'item_name': 'Shattered ...
31866    [{'item_id': '10', 'item_name': 'Counter-Strik...
Name: items, Length: 1357, dtype: object

In [15]:
#Eliminamos los valores duplicados
df_user_items = df_user_items.drop_duplicates(subset='user_id', keep='first')

#Verificamos si aún
verifica_duplicados(df_user_items, 'user_id')

'No hay duplicados'

#### 2.3.3. Transformación y eliminación de datos en la columna "items"

In [16]:
"""
Decidimos normalizar la columnas "items" debido a que hacer un explode y hacer una lista para guardar las filas al desanidar el diccionario puede
Provocar uso excesivo de memoria ese es el motivo por el cual decidimos normalizar.
"""
#Pasamos a normalizar la columna "items"
df_user_items_2 = pd.json_normalize(filas_items, record_path=['items'], meta=['user_id', 'items_count', 'steam_id', 'user_url'] )
df_user_items_2

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [17]:
#Verificamos si hay valores nulos en nuestro DataFrame luego de hacer la explosión
valores_nulos_df(df_user_items)

,nombre,tipo_datos,nulos_%,nulos
0,user_id,[<class 'str'>],0.0,0
1,items_count,[<class 'int'>],0.0,0
2,steam_id,[<class 'str'>],0.0,0
3,user_url,[<class 'str'>],0.0,0
4,items,[<class 'list'>],0.0,0


In [18]:
#Verificamos si existen duplicados en el nuevo DataFrame a diferencia de nuestra funcion de detectar duplicados en una columna este es para todo el DataFrame
duplicados = df_user_items_2.loc[df_user_items_2.duplicated()]
duplicados

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
164294,20,Team Fortress Classic,5,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
164295,50,Half-Life: Opposing Force,0,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
164296,70,Half-Life,0,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
164297,130,Half-Life: Blue Shift,0,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
164298,220,Half-Life 2,198,0,Nikiad,109,76561198084006094,http://steamcommunity.com/id/Nikiad
...,...,...,...,...,...,...,...,...
4898223,213670,South Park‚Ñ¢: The Stick of Truth‚Ñ¢,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


In [19]:
#Se observa que existen 59104 filas de valores duplicados en todo el DataFrame, los eliminamos
df_user_items_2 = df_user_items_2.drop_duplicates(keep='first')

In [20]:
#Visualizar las columnas que tiene nuestra nueva DataFrame
df_user_items_2.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id', 'items_count', 'steam_id', 'user_url'],
      dtype='object')

In [21]:
#Visualizamos las caracteristicas de nuestro nuevo DataFrame
caracteristicas_df(df_user_items_2)

**********||||||||||FORMA DE BASE DE DATOS||||||||||**********

Tiene 5094105 filas y 8 columnas o variables


**********||||||||||INFORMACION GENERAL DE LA BASE DE DATOS||||||||||**********

<class 'pandas.core.frame.DataFrame'>
Index: 5094105 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   user_id           object
 5   items_count       object
 6   steam_id          object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 349.8+ MB
None



In [22]:
#Mostramos el nuevo DataFrame
df_user_items_2.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


### 2.4. Exportación de DataSet Limpia

In [24]:
#Se guarda el dataframe transformado como australian_user_items_clean
archivo_limpio = r'Dataset_Clean\australian_user_items_clean.csv'
df_user_items_2.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

Se guardó el archivo Dataset_Clean\australian_user_items_clean.csv


In [25]:
#Se guarda el dataframe transformado como australian_user_items_clean
archivo_limpio_csv = r'Dataset_Clean\australian_user_items_clean.csv'
archivo_limpio_parquet = r'Dataset_Clean\australian_user_items_clean.parquet'
# Leemos el archivo CSV en un DataFrame
df = pd.read_csv(archivo_limpio_csv)
df.to_parquet(archivo_limpio_parquet, engine='pyarrow')
print(f'Se guardó el archivo {archivo_limpio_parquet}')

Se guardó el archivo Dataset_Clean\australian_user_items_clean.parquet
